In [1]:
! pip install pandas evaluate jiwer
! pip install tibetan_wer

Defaulting to user installation because normal site-packages is not writeable
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.9.0
    Uninstalling fsspec-2025.9.0:
      Successfully uninstalled fsspec-2025.9.0

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [2]:
def calculate_cer(row):
    try:
        uni = row['uni']
        inf_uni = row['tibetan_transcript']
        cer = cer_metric.compute(references=[uni], predictions=[inf_uni])
        cer = min(cer, 1.0)
        return cer
    except:
        return 0.0

In [25]:
from huggingface_hub import notebook_login

notebook_login()


In [26]:
# Script to download dataset from Hugging Face and merge with existing CSV

import os
import pandas as pd
from datasets import load_dataset

# Step 1: Download dataset from Hugging Face
print("Downloading dataset from Hugging Face...")
dataset = load_dataset("ganga4364/garchen_rinpoche_evaluation_results")

Generating train split:   0%|          | 0/893 [00:00<?, ? examples/s]

In [30]:
# Convert the train split to a pandas DataFrame
df = pd.DataFrame(dataset['train'])

# Display the first few rows
df.head()

,file_name,path,url,audio_duration,original_id,strata,age_group,duration_category,content_type,uni,...,ft_v7_scratch_lev_dist_23000,ft_v8_scartch_inference_checkpoint-10000,ft_v8_scratch_cer_10000,ft_v8_scratch_lev_dist_10000,ft_v8_scartch_inference_checkpoint-55000,ft_v8_scratch_cer_55000,ft_v8_scratch_lev_dist_55000,ft_v8_scartch_inference_checkpoint-59000,ft_v8_scratch_cer_59000,ft_v8_scratch_lev_dist_59000
0,STT_GR_0001_0010_93600_to_106200,./downloads/STT_GR_0001_0010_93600_to_106200.wav,https://d38pmlk0v88drf.cloudfront.net/wav16k/S...,12.600,STT_GR_0001,70-80__short__Teaching,70-80,short,Teaching,ཁོང་དེ་ནས་དགེ་འདུན་ཁྲི་ལ་སྔོན་ཟླ་རེ་བོད་གླེང་ད...,...,41,ཁོང། དེ་ནས་དགེ་དུས་ཁྲི་ལའི་རྔབ་བདག་རེད། བུ་གླ...,0.307263,57,ཁོང་། དེ་ནས་དགེ་འདུན་ཁྲིི་ང་ད་རེད། བུ་གླིང་དེ...,0.262570,49,ཁོང་་དེ་ས་དགེ་འདུལ་ཁྲིི་ལང་བདག་རེད། བུ་གླིང་དེ...,0.290503,54
1,STT_GR_0033_0105_224320_to_225856,./downloads/STT_GR_0033_0105_224320_to_225856.wav,https://d38pmlk0v88drf.cloudfront.net/wav16k/S...,1.536,STT_GR_0033,70-80__short__Teaching,70-80,short,Teaching,ཕ་གི་ཕར་རང་རེ་རྫས་ཚར་ལེ།,...,18,ཨ། ཕན་པ་འརེད་ཡོད་རེ། དེ་ལདྟ་ལུ།,0.833333,20,ཕན་པ་ཡོད་ན་དེ་ལྟ་ལུ་,0.708333,17,ཨ། ཕན་པ་ཡོད་ལ་དེ་ལྟ་ལུ།,0.750000,18
2,STT_GR_0032_0004_19067_to_24988,./downloads/STT_GR_0032_0004_19067_to_24988.wav,https://d38pmlk0v88drf.cloudfront.net/wav16k/S...,5.921,STT_GR_0032,70-80__short__Teaching,70-80,short,Teaching,དབང་པོ་གསལ་མཁན་དེ་ཡིས་དབང་སྐུར་དེ་ལ་གསང་རྫས་མཁ...,...,52,སོན་པོ་དེ་ཡིན་ད་ཡང་ཨ་ སྐུ་ལ་འད་རབ་བསམས་། དེ་མག...,0.557692,58,ཕིམ་པ་དེ་ཞི་ནཡང་ཨ་གུ་འདི་ལ་སམས་འད་སགོན་འགྲོ་རོ...,0.596154,62,ཕུ་པུ་དེ་གིས་ལ་ཡུང་བ་གུན་འདི་ལ་སག་འད་སགོུན་འགྲ...,0.538462,56
3,STT_GR_0033_0099_212224_to_213024,./downloads/STT_GR_0033_0099_212224_to_213024.wav,https://d38pmlk0v88drf.cloudfront.net/wav16k/S...,0.800,STT_GR_0033,70-80__short__Teaching,70-80,short,Teaching,ཐབས་དང་ཟབ་ཅིང་,...,9,ཐ་མ་སྔས་རདད་ཀྱ།,0.857143,12,ཐའ་མ་ཐ་ཅིག།,0.571429,8,ཐ་པ་ཐམས་ཅིད་,0.571429,8
4,STT_GR_0033_0025_56251_to_58684,./downloads/STT_GR_0033_0025_56251_to_58684.wav,https://d38pmlk0v88drf.cloudfront.net/wav16k/S...,2.433,STT_GR_0033,70-80__short__Teaching,70-80,short,Teaching,ཁོ་ཚོས་གཅིག་གོ་ཐུབ་ཀི་འདུག ཡག་པོ་འདུག་བསམ་གི་འ...,...,4,ཁོ་དོས་་གོ་ཐུབ་ཀྱི་འདུ་། ཡག་པོ་འདུག་བསམ་གི་འདུག་,0.204082,10,ཁོ་ཚོ་ཕཞིག་གོ་ཐུབ་ཀྱི་འདུག ཡག་པོ་འདུག་བསམ་གྱི་...,0.102041,6,ཁོ་དུ་གཅདེག་ལ་གོ་ཐུབ་ཀྱི་འདུག ཡག་པོ་འདུག་གསམ་ག...,0.204082,11


In [31]:
df.columns

Index(['file_name', 'path', 'url', 'audio_duration', 'original_id', 'strata',
       'age_group', 'duration_category', 'content_type', 'uni',
       'base_transcript', 'base_cer', 'base_lev_dist',
       'ft_v1_inference_checkpoint-5000', 'ft_v1_cer_5000',
       'ft_v1_lev_dist_5000', 'ft_v1_inference_checkpoint-10000',
       'ft_v1_cer_10000', 'ft_v1_lev_dist_10000',
       'ft_v1_inference_checkpoint-19000', 'ft_v1_cer_19000',
       'ft_v1_lev_dist_19000', 'ft_v2_inference_checkpoint-20000',
       'ft_v2_cer_20000', 'ft_v2_lev_dist_20000',
       'ft_v2_inference_checkpoint-32000', 'ft_v2_cer_32000',
       'ft_v2_lev_dist_32000', 'ft_v2_inference_checkpoint-43000',
       'ft_v2_cer_43000', 'ft_v2_lev_dist_43000',
       'ft_v3_inference_checkpoint-1000', 'ft_v3_cer_1000',
       'ft_v3_lev_dist_1000', 'ft_v3_inference_checkpoint-25000',
       'ft_v3_cer_25000', 'ft_v3_lev_dist_25000',
       'ft_v4_inference_checkpoint-22000', 'ft_v4_cer_22000',
       'ft_v4_lev_dist_22000', 

In [32]:
import pandas as pd

df1 = pd.read_csv("/home/gangagyatso/Downloads/output.csv")



In [36]:
df1 = df1[['file_name', 'tibetan_transcript', 'wylie_transcript', 'wylie_to_tibetan_text']]

In [37]:
# Basic merge on file_name column
merged_df = pd.merge(df, df1, on='file_name')

# Display the first few rows of the merged DataFrame
merged_df.head()

,file_name,path,url,audio_duration,original_id,strata,age_group,duration_category,content_type,uni,...,ft_v8_scratch_lev_dist_10000,ft_v8_scartch_inference_checkpoint-55000,ft_v8_scratch_cer_55000,ft_v8_scratch_lev_dist_55000,ft_v8_scartch_inference_checkpoint-59000,ft_v8_scratch_cer_59000,ft_v8_scratch_lev_dist_59000,tibetan_transcript,wylie_transcript,wylie_to_tibetan_text
0,STT_GR_0001_0010_93600_to_106200,./downloads/STT_GR_0001_0010_93600_to_106200.wav,https://d38pmlk0v88drf.cloudfront.net/wav16k/S...,12.600,STT_GR_0001,70-80__short__Teaching,70-80,short,Teaching,ཁོང་དེ་ནས་དགེ་འདུན་ཁྲི་ལ་སྔོན་ཟླ་རེ་བོད་གླེང་ད...,...,57,ཁོང་། དེ་ནས་དགེ་འདུན་ཁྲིི་ང་ད་རེད། བུ་གླིང་དེ...,0.262570,49,ཁོང་་དེ་ས་དགེ་འདུལ་ཁྲིི་ལང་བདག་རེད། བུ་གླིང་དེ...,0.290503,54,ཁོང་དེ་ནས་དགེ་དུས་འཕྲལ་པའི་ལྔ་ཟླ་རེད། དེ་བུ་གླ...,khong de nas dge du khri ba'i ngab da re yin ...,ཁོང་དེ་ནས་དགེ་དུ་ཁྲི་བའི་ངབ་ད་རེ་ཡིན་པོཨོན་གླི...
1,STT_GR_0033_0105_224320_to_225856,./downloads/STT_GR_0033_0105_224320_to_225856.wav,https://d38pmlk0v88drf.cloudfront.net/wav16k/S...,1.536,STT_GR_0033,70-80__short__Teaching,70-80,short,Teaching,ཕ་གི་ཕར་རང་རེ་རྫས་ཚར་ལེ།,...,20,ཕན་པ་ཡོད་ན་དེ་ལྟ་ལུ་,0.708333,17,ཨ། ཕན་པ་ཡོད་ལ་དེ་ལྟ་ལུ།,0.750000,18,ཨ། ཕན་པ་རེད་གོ་ན་དེ་ལྟར་ནས།,phan phan ba lag gi yod na de lta bu la,ཕན་ཕན་བ་ལག་གི་ཡོད་ན་དེ་ལྟ་བུ་ལ
2,STT_GR_0032_0004_19067_to_24988,./downloads/STT_GR_0032_0004_19067_to_24988.wav,https://d38pmlk0v88drf.cloudfront.net/wav16k/S...,5.921,STT_GR_0032,70-80__short__Teaching,70-80,short,Teaching,དབང་པོ་གསལ་མཁན་དེ་ཡིས་དབང་སྐུར་དེ་ལ་གསང་རྫས་མཁ...,...,58,ཕིམ་པ་དེ་ཞི་ནཡང་ཨ་གུ་འདི་ལ་སམས་འད་སགོན་འགྲོ་རོ...,0.596154,62,ཕུ་པུ་དེ་གིས་ལ་ཡུང་བ་གུན་འདི་ལ་སག་འད་སགོུན་འགྲ...,0.538462,56,བུ་དེས་ང་དང་ཡང་བ་གོལ་དུ་དེ་ལ་གསུམ། དེ་ཀུན་འགྲོ...,po tshig nga la yang ba yang kho ru de la gsu...,པོ་ཚིག་ང་ལ་ཡང་བ་ཡང་ཁོ་རུ་དེ་ལ་གསུང་དང་ཁོང་ཆུ་ད...
3,STT_GR_0033_0099_212224_to_213024,./downloads/STT_GR_0033_0099_212224_to_213024.wav,https://d38pmlk0v88drf.cloudfront.net/wav16k/S...,0.800,STT_GR_0033,70-80__short__Teaching,70-80,short,Teaching,ཐབས་དང་ཟབ་ཅིང་,...,12,ཐའ་མ་ཐ་ཅིག།,0.571429,8,ཐ་པ་ཐམས་ཅིད་,0.571429,8,མཚམས་མའི་ཐམས་ཅད་,thab dang thabs te,ཐབ་དང་ཐབས་ཏེ
4,STT_GR_0033_0025_56251_to_58684,./downloads/STT_GR_0033_0025_56251_to_58684.wav,https://d38pmlk0v88drf.cloudfront.net/wav16k/S...,2.433,STT_GR_0033,70-80__short__Teaching,70-80,short,Teaching,ཁོ་ཚོས་གཅིག་གོ་ཐུབ་ཀི་འདུག ཡག་པོ་འདུག་བསམ་གི་འ...,...,10,ཁོ་ཚོ་ཕཞིག་གོ་ཐུབ་ཀྱི་འདུག ཡག་པོ་འདུག་བསམ་གྱི་...,0.102041,6,ཁོ་དུ་གཅདེག་ལ་གོ་ཐུབ་ཀྱི་འདུག ཡག་པོ་འདུག་གསམ་ག...,0.204082,11,ཁོ་ཚོས་གཅིག་གི་གོ་ཐུབ་ཀྱི་འདུག ཡག་པོ་འདུག་བསམ་...,kho tshos gcig rtsa kho thub mga'i nang du ya...,ཁོ་ཚོས་གཅིག་རྩ་ཁོ་ཐུབ་མགའི་ནང་དུ་ཡག་པོ་རྡུག་བས...


In [38]:
len(merged_df)

893

In [40]:
merged_df.to_csv('merged_df.csv', index=False)

In [39]:
merged_df.columns

Index(['file_name', 'path', 'url', 'audio_duration', 'original_id', 'strata',
       'age_group', 'duration_category', 'content_type', 'uni',
       'base_transcript', 'base_cer', 'base_lev_dist',
       'ft_v1_inference_checkpoint-5000', 'ft_v1_cer_5000',
       'ft_v1_lev_dist_5000', 'ft_v1_inference_checkpoint-10000',
       'ft_v1_cer_10000', 'ft_v1_lev_dist_10000',
       'ft_v1_inference_checkpoint-19000', 'ft_v1_cer_19000',
       'ft_v1_lev_dist_19000', 'ft_v2_inference_checkpoint-20000',
       'ft_v2_cer_20000', 'ft_v2_lev_dist_20000',
       'ft_v2_inference_checkpoint-32000', 'ft_v2_cer_32000',
       'ft_v2_lev_dist_32000', 'ft_v2_inference_checkpoint-43000',
       'ft_v2_cer_43000', 'ft_v2_lev_dist_43000',
       'ft_v3_inference_checkpoint-1000', 'ft_v3_cer_1000',
       'ft_v3_lev_dist_1000', 'ft_v3_inference_checkpoint-25000',
       'ft_v3_cer_25000', 'ft_v3_lev_dist_25000',
       'ft_v4_inference_checkpoint-22000', 'ft_v4_cer_22000',
       'ft_v4_lev_dist_22000', 

In [4]:
df.columns

Index(['file_name', 'url', 'audio_duration', 'original_id', 'path', 'strata',
       'age_group', 'duration_category', 'content_type', 'uni',
       'base_transcript', 'base_cer', 'ft_v1_inference_checkpoint-5000',
       'ft_v1_cer_5000', 'ft_v1_inference_checkpoint-10000', 'ft_v1_cer_10000',
       'ft_v1_inference_checkpoint-19000', 'ft_v1_cer_19000',
       'ft_v2_inference_checkpoint-20000', 'ft_v2_cer_20000',
       'ft_v2_inference_checkpoint-32000', 'ft_v2_cer_32000',
       'ft_v2_inference_checkpoint-43000', 'ft_v2_cer_43000',
       'ft_v3_inference_checkpoint-1000', 'ft_v3_cer_1000',
       'ft_v3_inference_checkpoint-25000', 'ft_v3_cer_25000',
       'ft_v4_inference_checkpoint-22000', 'ft_v4_cer_22000',
       'tibetan_transcript', 'wylie_transcript', 'wylie_to_tibetan_text'],
      dtype='object')

In [5]:
df.head()

,file_name,url,audio_duration,original_id,path,strata,age_group,duration_category,content_type,uni,...,ft_v2_cer_43000,ft_v3_inference_checkpoint-1000,ft_v3_cer_1000,ft_v3_inference_checkpoint-25000,ft_v3_cer_25000,ft_v4_inference_checkpoint-22000,ft_v4_cer_22000,tibetan_transcript,wylie_transcript,wylie_to_tibetan_text
0,STT_GR_0001_0010_93600_to_106200,https://d38pmlk0v88drf.cloudfront.net/wav16k/S...,12.600,STT_GR_0001,./downloads/STT_GR_0001_0010_93600_to_106200.wav,70-80__short__Teaching,70-80,short,Teaching,ཁོང་དེ་ནས་དགེ་འདུན་ཁྲི་ལ་སྔོན་ཟླ་རེ་བོད་གླེང་ད...,...,0.245810,ཁོང། དེ་ནས་དགེ་འདུན་ཁྲིའི་ལྔ་བདག་རེད། བུ་གླིང...,0.273743,ཁོང། དེ་ནས་དགེ་འདུན་ཁྲིི་ལྔ་དག་རེད། བུ་གླིང་ད...,0.279330,ཁོང་། དེ་ནས་དགེ་འདུན་ཁྲིའི་ལྔ་བདག་རེད། བུ་གླི...,0.273743,ཁོང་དེ་ནས་དགེ་དུས་འཕྲལ་པའི་ལྔ་ཟླ་རེད། དེ་བུ་གླ...,khong de nas dge du khri ba'i ngab da re yin ...,ཁོང་དེ་ནས་དགེ་དུ་ཁྲི་བའི་ངབ་ད་རེ་ཡིན་པོཨོན་གླི...
1,STT_GR_0033_0105_224320_to_225856,https://d38pmlk0v88drf.cloudfront.net/wav16k/S...,1.536,STT_GR_0033,./downloads/STT_GR_0033_0105_224320_to_225856.wav,70-80__short__Teaching,70-80,short,Teaching,ཕ་གི་ཕར་རང་རེ་རྫས་ཚར་ལེ།,...,0.833333,ཨ་ཕན་པ་རེད་ཡོད་རེད་དེ་ྟེན་ནུ་,0.833333,ཨ། ཕན་པ་ྲ་ཡོད་ན་དེ་ལྟ་བུ་,0.875000,ཕན་པ་རེད་ཡོད་ན་དེ་ལྟ་ད,0.750000,ཨ། ཕན་པ་རེད་གོ་ན་དེ་ལྟར་ནས།,phan phan ba lag gi yod na de lta bu la,ཕན་ཕན་བ་ལག་གི་ཡོད་ན་དེ་ལྟ་བུ་ལ
2,STT_GR_0032_0004_19067_to_24988,https://d38pmlk0v88drf.cloudfront.net/wav16k/S...,5.921,STT_GR_0032,./downloads/STT_GR_0032_0004_19067_to_24988.wav,70-80__short__Teaching,70-80,short,Teaching,དབང་པོ་གསལ་མཁན་དེ་ཡིས་དབང་སྐུར་དེ་ལ་གསང་རྫས་མཁ...,...,0.432692,ཀོན་པ་དེང་དི་ཡལ་བ་ྐུ་ི་ཏི་ར་སེ་དི་སྔོན་འགྲོ་ལྔ...,0.471154,ཨན་པ་དེ་གས་ནི་ཡང་བང་མཁོས་འདི་ལ་གསམ། འདི་ མཁོ་འ...,0.509615,བམི་དེ་གས་ན་ཡང་དབ་སྐོར་ར་འད་ལ་གསིཾ་ཚེ་སཁོར་འགྲ...,0.490385,བུ་དེས་ང་དང་ཡང་བ་གོལ་དུ་དེ་ལ་གསུམ། དེ་ཀུན་འགྲོ...,po tshig nga la yang ba yang kho ru de la gsu...,པོ་ཚིག་ང་ལ་ཡང་བ་ཡང་ཁོ་རུ་དེ་ལ་གསུང་དང་ཁོང་ཆུ་ད...
3,STT_GR_0033_0099_212224_to_213024,https://d38pmlk0v88drf.cloudfront.net/wav16k/S...,0.800,STT_GR_0033,./downloads/STT_GR_0033_0099_212224_to_213024.wav,70-80__short__Teaching,70-80,short,Teaching,ཐབས་དང་ཟབ་ཅིང་,...,0.642857,ཐམ་པ་ཐ་ཞ་་,0.642857,ཐག་མ་ཐས་དད་,0.642857,ཐམ་མ་ཐས་ཀི་,0.571429,མཚམས་མའི་ཐམས་ཅད་,thab dang thabs te,ཐབ་དང་ཐབས་ཏེ
4,STT_GR_0033_0025_56251_to_58684,https://d38pmlk0v88drf.cloudfront.net/wav16k/S...,2.433,STT_GR_0033,./downloads/STT_GR_0033_0025_56251_to_58684.wav,70-80__short__Teaching,70-80,short,Teaching,ཁོ་ཚོས་གཅིག་གོ་ཐུབ་ཀི་འདུག ཡག་པོ་འདུག་བསམ་གི་འ...,...,0.102041,ཁོ་ཚོས་གཅིག་གི་གོ་ཐུབ་ཀྱི་འདུག ཡག་པོ་འདུག་བསམ་...,0.102041,ཁོ་ཚོས་གཅིག་གི་གོ་ཐུབ་ཀྱི་འདུག ཡག་པོ་འདུག་བསམ་...,0.102041,ཁོ་ཚོས་གཅིག་གི་གོ་ཐུབ་ཀྱི་འདུག ཡག་པོ་འདུག་བསམ་...,0.102041,ཁོ་ཚོས་གཅིག་གི་གོ་ཐུབ་ཀྱི་འདུག ཡག་པོ་འདུག་བསམ་...,kho tshos gcig rtsa kho thub mga'i nang du ya...,ཁོ་ཚོས་གཅིག་རྩ་ཁོ་ཐུབ་མགའི་ནང་དུ་ཡག་པོ་རྡུག་བས...


In [6]:
from tibetan_wer.metrics import wer, ser

Loading Trie... (1s.)


In [7]:
from datasets import cer_metric

references = list(df['uni'])
predictions = list(df['tibetan_transcript'])

result = cer_metric.compute(predictions=predictions, references=references)
print(result)


ImportError: cannot import name 'cer_metric' from 'datasets' (/home/gangagyatso/.local/lib/python3.10/site-packages/datasets/__init__.py)

In [8]:
! pip install jiwer evaluate

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [9]:
from evaluate import load
cer_metric = load("cer")

Using the latest cached version of the module from /home/gangagyatso/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--cer/9cb90b752d5f15fb41161efdbefd13570adb3f32fa157290d8a55093c47428e1 (last modified on Fri Jun  7 15:35:04 2024) since it couldn't be found locally at evaluate-metric--cer, or remotely on the Hugging Face Hub.


In [10]:
rand_index = 5
print(df.loc[rand_index, 'uni'])
print(df.loc[rand_index, 'tibetan_transcript'])

wer = wer(df.loc[rand_index, 'uni'], df.loc[rand_index, 'tibetan_transcript'])

print(wer)

དེའི་རྩིས་གཞི་ད་ད་ལྟ་སྐད་ཆ་བཤད་དགོས་རེད་ད། དྲན་པས་ཡང་མི་གནོད།
དེ་ནས་དེ་དུས། ད་ད་ལྟ་སྐད་ཆ་བཤད་དགོས་རེད། ཨ་དྲན་གྱི་ཡང་མི་འདུག
{'micro_wer': 0.4915254237288136, 'macro_wer': np.float64(0.4576271186440678), 'substitutions': 26, 'insertions': 2, 'deletions': 1, 'num_sentences': 61}


In [11]:
from tibetan_wer.metrics import wer

prediction = ['གཞོན་ནུར་གྱུར་པ་ལ་ཕྱག་འཚལ་ལོ༔', 'དེ་ནས་ད་ལྟ་ཀར་སྤང་དེ་ལ་དེ་ནས། ཨ།']
reference = ['འཇམ་དཔལ་གཞོན་ནུར་གྱུར་པ་ལ་ཕྱག་འཚལ་ལོ༔', 'དེ་ནས་ད་ལྟ་ཀར་སྤང་དེ་ལ་དེ་ནས། ཨ།']

result = wer(prediction, reference)
print(result)


{'micro_wer': 0.05263157894736842, 'macro_wer': np.float64(0.05555555555555555), 'substitutions': 0, 'insertions': 0, 'deletions': 1, 'num_sentences': 2}


In [12]:
from tibetan_wer.metrics import ser

prediction = ['གཞོན་ནུར་གྱུར་པ་ལ་ཕྱག་འཚལ་ལོ༔']
reference = ['འཇམ་དཔལ་གཞོན་ནུར་གྱུར་པ་ལ་ཕྱག་འཚལ་ལོ༔']

result = ser(prediction, reference)

print(f'Micro-Average SER Score: {result["micro_ser"]:.3f}')
print(f'Macro-Average SER Score: {result["macro_ser"]:.3f}')
print(f'Substitutions: {result["substitutions"]:.3f}')
print(f'Insertions: {result["insertions"]:.3f}')
print(f'Deletions: {result["deletions"]:.3f}')

Micro-Average SER Score: 0.200
Macro-Average SER Score: 0.200
Substitutions: 0.000
Insertions: 0.000
Deletions: 2.000


In [ ]:
df['cer'] = df.apply(lambda row: calculate_cer(row), axis=1)


In [ ]:
df['cer'].mean()

In [18]:
len(df)

893

In [19]:
references = list(df['uni'])
predictions = list(df['wylie_to_tibetan_text'])


In [20]:
cer = cer_metric.compute(predictions=predictions, references=references)
print(cer)

0.3984519498282535


In [21]:
wer_result = wer(predictions, references)
print(wer_result)

{'micro_wer': 0.6753966736761613, 'macro_wer': np.float64(0.7124242870608974), 'substitutions': 6377, 'insertions': 922, 'deletions': 3300, 'num_sentences': 893}


In [22]:
ser_result =  ser(predictions, references)
print(ser_result)


{'micro_ser': 0.6165623670939911, 'macro_ser': np.float64(0.6562800512792923), 'substitutions': 6741, 'insertions': 1561, 'deletions': 1846, 'num_sentences': 893}


In [17]:
df['audio_duration'].sum()/60/60


np.float64(1.0750530555555557)

In [ ]:
len(df)

In [ ]:
import os
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from multiprocessing import Pool, cpu_count
import requests
import os
import requests
import logging

# Configure logging
logging.basicConfig(filename='failed_downloads.log', level=logging.ERROR, format='%(asctime)s - %(message)s')

# Function to download audio file
def download_audio(url, file_name, save_path):
    # Full path where the audio will be saved
    
    # Skip download if file already exists
    if os.path.exists(save_path):
        return
    
    try:
        # Download the file from the URL and save it to the specified path
        response = requests.get(url, timeout=10)
        response.raise_for_status()  # Check for HTTP errors
        with open(save_path, 'wb') as f:
            f.write(response.content)
    except requests.exceptions.RequestException as e:
        # Log the error and file name if the download fails
        logging.error(f"Failed to download {file_name}: {e}")


# Function to download the audio file and perform inference
def process_row(row):
    url = row['url']
    file_name = os.path.basename(url)
    
    # Create the save path for the temporary audio file
    save_path = f"../data/wav16k/{file_name}"
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    
    # Function to download the audio file
    download_audio(url, file_name, save_path)
    
    return row

# Function to process the DataFrame using Pool
def process_dataframe(df):
    # Number of processes to run, set to the number of available CPU cores or adjust as needed
    num_workers = min(cpu_count(), 5)  # Adjust based on your system capabilities

    # Use Pool to process rows in parallel
    with Pool(processes=num_workers) as pool:
        rows = list(tqdm(pool.imap(process_row, [row for _, row in df.iterrows()]), total=len(df)))

    # Convert the processed rows back into a DataFrame
    return pd.DataFrame(rows)



if __name__ == "__main__":

    df = pd.read_csv("sr_submitted.csv")
    # Add required columns
    df['Path'] = '../data/wav16k/' + df['file_name'] + ".wav"

    processed_df = process_dataframe(df)
   

In [ ]:
import os
import sys
import subprocess

sys.path.append('../util')


from common_utils import get_time_span
from common_utils import clean_transcription
from transformers import pipeline
from tqdm.auto import tqdm
import pandas as pd
from pathlib import Path
from common_utils import parse_args_and_load_config



# Read configuration from the loaded JSON
dept = 'GR'
target_path = Path('../data/wav16k/')
rows = []

generator = pipeline(model="openpecha/Situ_rinpoche_asr")

for file in tqdm(target_path.glob('*.wav'), total=len(list(target_path.glob('*.wav')))):
    inf = generator(str(file))["text"]
    rows.append([file.stem, f"https://d38pmlk0v88drf.cloudfront.net/wav16k/{file.name}", inf, get_time_span(str(file.name))])


df = pd.DataFrame(rows, columns =['file_name', 'url', 'inference_transcript', 'audio_duration'])
df = df.sort_values('file_name').reset_index(drop=True)
df[['inference_transcript','url']].iloc[0:10].to_dict()
df['inference_transcript'] = df['inference_transcript'].map(clean_transcription)
df.to_csv(f"../data/gr_situ1.csv", index=False)

   

In [ ]:
df1 = pd.read_csv('../data/gr_situ1.csv')
df2 = pd.read_csv('sr_submitted.csv')

In [ ]:
df1.head()

In [ ]:
df2.head()

In [ ]:
# If you want to assign values from df2['transcript'] to df1['transcript'] where file_name matches:
df1['transcript'] = df1.apply(lambda row: df2.loc[df2['file_name'] == row['file_name'], 'transcript'].values[0] 
                             if any(df2['file_name'] == row['file_name']) else row['transcript'], axis=1)

In [ ]:
df1.head()

In [ ]:
df1.to_csv('gr_situ2.csv', index=False)

In [ ]:
import pandas as pd

df1 = pd.read_csv("sr_submitted.csv")

In [ ]:
df2 = pd.read_csv("gr_situ2.csv")

In [ ]:
df3 = pd.read_csv("gr_dilgo1.csv")

In [ ]:
df3.head()

In [ ]:
# Assuming df1 and df3 are already created dataframes
# This will add the inference_transcript from df1 to df3 where file_name matches
df3["General_model_inference"] = df3["file_name"].map(df1.set_index("file_name")["inference_transcript"])

In [ ]:
df3["situ_rinpoche_model_inference"] = df3["file_name"].map(df2.set_index("file_name")["inference_transcript"])

In [ ]:
df3.head()

In [ ]:
df3.rename(columns={'inference_transcript':'Dilgo_kyentse_Rinpoche_model_inference'}, inplace=True)

In [ ]:
df3.head()

In [ ]:
df3.rename(columns={'transcript':'Human_transcript'}, inplace=True)

In [ ]:
df3.head()

In [ ]:
from huggingface_hub import login
login()  # This will prompt for your Hugging Face token

In [ ]:
import pandas as pd 

# Load the CSV file into a pandas DataFrame
df = pd.read_csv('processed_test_with_cer.csv')

In [ ]:

# Print the first few rows of the DataFrame
df.head()

In [ ]:
df.columns

In [ ]:
# Reorder columns into logical groups
reordered_df = df[
    # File and metadata columns
    ['file_name', 'url',  'audio_duration', 'original_id', 'path',
     'strata', 'age_group', 'duration_category', 'content_type', 'uni',
    
    # Base model
    'base_transcript', 'base_cer',
    
    # v1 models (grouped together - both inferences and CERs)
    'ft_v1_inference_checkpoint-5000', 'ft_v1_cer_5000',
    'ft_v1_inference_checkpoint-10000', 'ft_v1_cer_10000',
    'ft_v1_inference_checkpoint-19000', 'ft_v1_cer_19000',
    
    # v2 models
    'ft_v2_inference_checkpoint-20000', 'ft_v2_cer_20000',
    'ft_v2_inference_checkpoint-32000', 'ft_v2_cer_32000',
    'ft_v2_inference_checkpoint-43000', 'ft_v2_cer_43000',
    
    # v3 models
    'ft_v3_inference_checkpoint-1000', 'ft_v3_cer_1000',
    'ft_v3_inference_checkpoint-25000', 'ft_v3_cer_25000',
    
    # v4 model
    'ft_v4_inference_checkpoint-22000', 'ft_v4_cer_22000'
    ]
]

In [ ]:
from datasets import Dataset
from huggingface_hub import HfApi

# Assuming df2 is a pandas DataFrame
# Convert your pandas DataFrame to a Hugging Face Dataset
hf_dataset = Dataset.from_pandas(reordered_df)

# Create a new repository on the Hub
api = HfApi()
repo_id = "ganga4364/garchen_rinpoche_inference_transcripts"  # replace with your username and desired dataset name

# Push the dataset to the Hub
hf_dataset.push_to_hub(repo_id)